<a href="https://colab.research.google.com/github/dannsb/neural-network/blob/main/Next_Word_Prediction(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import libs

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense,Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
file = open("data.txt","r",encoding="utf8")

### pre proccessing on text data

In [ ]:
lines = []
for i in file:
  lines.append(i)

In [ ]:
data=""
for i in lines:
    data = ' '.join(lines)
data = data.replace("\n","").replace("\r","").replace("ufeff","").replace('“','').replace('”','')

In [ ]:
data = data.split()
data = " ".join(data)

Tokenizer: creates the vocabulary index based on word frequency

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

Pickle “serializes” the object first before writing it to a file. Pickling is a way to convert a Python object (list, dictionary, etc.) into a character stream.

In [ ]:
pickle.dump(tokenizer, open("token.pkl","wb"))

In [ ]:
sequenceData = tokenizer.texts_to_sequences([data])[0]

In [ ]:
sequence=[]
for i in range(3, len(sequenceData)):
    words = sequenceData[i-3:i+1]
    sequence.append(words)

In [ ]:
sequence = np.array(sequence)

In [ ]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
vocabSize = len(tokenizer.word_index)+1
y = to_categorical(y, vocabSize)

### add model & fit

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocabSize, output_dim=10))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocabSize, activation="softmax"))

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("nextword.keras", monitor='loss', verbose=1,
    save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, epochs=2, batch_size=64, callbacks=[checkpoint]) #NOTE: epochs should be more

Epoch 1/2
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 6.6405
Epoch 1: loss improved from inf to 6.34087, saving model to nextword.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 49s 26ms/step - loss: 6.6403
Epoch 2/2
1653/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.8071
Epoch 2: loss improved from 6.34087 to 5.75648, saving model to nextword.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - loss: 5.8070


### predict function
besed on keras model file generated by RNN(LSTM).in my example: **nextword.keras**

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('nextword.keras')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def predict_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds=np.argmax(model.predict(sequence))
    predicted_word = ""
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    return predicted_word


predict_word(model, tokenizer, "A Scandal in")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step


'i'